In [1]:
import numpy
import cv2 as cv
import sys

cv2 = [opencv-python](https://pypi.org/project/opencv-python/) 4.4.0.46

opencv-python is a python wrapper for cv2, which runs on c++ code, allowing for code that is both easy and fast.

cv2 creates numpy arrays, so importing numpy may come in handy.

## Table of contents

[Opening an image with cv2 and optionally saving](#Opening-an-image-with-cv2-and-optionally-saving)

[Capturing an image from my laptop camera](#Capturing-an-image-from-my-laptop-camera)

[Capturing an image of just my hands via cropping, for now](#Capturing-an-image-of-just-my-hands-via-cropping,-for-now)

[Foreground detection](#Foreground-detection)

[Cropping based on hand presence](#Cropping-based-on-hand-presence)

## Opening an image with cv2 and optionally saving

Made using this [documentation tutorial](https://docs.opencv.org/master/db/deb/tutorial_display_image.html) and [this tutorial](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_image_display/py_image_display.html).

In [3]:
# read image from folder
# if image read works then imread() returns numpy array
# if image read fails then imread() returns None
img = cv.imread('data/self_photos/capture.png')  
if img is None:
    sys.exit("Could not read the image.")

# pops up a window named 'image' that has the image img
cv.imshow('image',img)  

# cv.waitKey(0) waits indefinitely for a key press,
# returning that pressed key as a string
k = cv.waitKey(0) 

# press 's' to save the image into the following directory with the following name
if k == ord("s"):
    cv.imwrite('data/opencv_images/capturecopy.png', img)

#closes all windows opened by cv
cv.destroyAllWindows()


## Capturing an image from my laptop camera

Made using [this stack overflow discussion](https://stackoverflow.com/questions/34588464/python-how-to-capture-image-from-webcam-on-click-using-opencv#34588758).

In [4]:
vid_feed = cv.VideoCapture(0) #  goes with the first camera it can find in this device

#  creates a new window titled by the input string
cv.namedWindow('test')
print('video feed launching')
while True:
    success_bool, frame = vid_feed.read()
    
    if (not success_bool):
        print('error reading video feed')
        break        
    
    cv.imshow('test', frame)
    # cv.waitKey(1) waits 1ms for a key press,
    # returning that pressed key as a string
    k = cv.waitKey(1) 

    # press 's' to save the image into the following directory with the following name
    if k == ord("s"):
        cv.imwrite('data/opencv_images/capturecopy.png', frame)
        print('image saved')
    elif k == ord("q"):
        print('quit')
        break
    else:
        pass

#closes all windows opened by cv
cv.destroyAllWindows()    
    

video feed launching
quit


## Capturing an image of just my hands via cropping, for now

As explained in [this stackexchange post](https://stackoverflow.com/questions/15589517/how-to-crop-an-image-in-opencv-using-python), cropping can be simply achieved by using numpy slicing.

As a quick fix, the frame will be set manually and the hand will be moved until it is in frame.

In [3]:
vid_feed = cv.VideoCapture(0) #  goes with the first camera it can find in this device

#  creates a new window titled by the input string
cv.namedWindow('test')
print('video feed launching')
while True:
    success_bool, frame = vid_feed.read()
    
    if (not success_bool):
        print('error reading video feed')
        break        
    
    #cropping frame
    frame = frame[150:420, 0:250] #  [ V , > ]
    
    cv.imshow('test', frame)
    # cv.waitKey(1) waits 1ms for a key press,
    # returning that pressed key as a string
    k = cv.waitKey(1) 

    # press 's' to save the image into the following directory with the following name
    if k == ord("s"):
        cv.imwrite('data/opencv_images/capturecopy.png', frame)
        print('image saved')
    elif k == ord("q"):
        print('quit')
        break
    else:
        pass

#closes all windows opened by cv
cv.destroyAllWindows()    
    

video feed launching
quit


## Foreground detection

By subtracting the background from images, the background may contribute less to CNN readings.

First try, subtracting an image of the background from the current image.
This may mess up the hand's pixel values a lot.

In [2]:
vid_feed = cv.VideoCapture(0) #  goes with the first camera it can find in this device

#  creates a new window titled by the input string
cv.namedWindow('test')
print('video feed launching')

success_bool, bg_frame = vid_feed.read()
    
if (not success_bool):
    print('error reading video feed')
    
while True:
    success_bool, frame = vid_feed.read()
    
    if (not success_bool):
        print('error reading video feed')
        break 
        
    #background subtraction
    frame = frame - bg_frame
        
    #cropping frame
    frame = frame[150:420, 0:250] #  [ V , > ]

    #showing image
    cv.imshow('test', frame)
    
    # cv.waitKey(1) waits 1ms for a key press,
    # returning that pressed key as a string
    k = cv.waitKey(1) 

    # press 's' to save the image into the following directory with the following name
    if k == ord("s"):
        cv.imwrite('data/opencv_images/capturecopy.png', frame)
        print('image saved')
    elif k == ord("q"):
        print('quit')
        break
    else:
        pass

#closes all windows opened by cv
cv.destroyAllWindows()    
    

video feed launching
quit


This was very cool looking. It made a print of the bookshelf appear along the silhouette of my hand, and multicolor noise filled the background, seeming to vary as the camera experienced noise and as it automatically adjusted its settings to maintain image quality.

Second try, background Subtraction via BackgroundSubtractorMOG().

In [18]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorMOG2(history = 50000,detectShadows=False)

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)

    cv2.imshow('frame',fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

This seems to generate hand outlines well, but parts of the hand as well as hand placement seem to become categorized as background over time, creating pixel blobs within the clear silhouette.

## Cropping based on hand presence
(possibly using machine learning models to crop hands well)